<a href="https://colab.research.google.com/github/Changyoon-Lee/mulcamB_1/blob/master/%ED%95%9C%EA%B5%AD%EC%9D%80%ED%96%89_%EC%9D%98%EC%82%AC%EB%A1%9D_%EC%BD%94%EB%93%9C_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Crawling을 활용하여 pdf파일을 다운 받을수 있는 url 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup

total_list = [] # 설정된 page의 tag list

for i in range(32,33): 
    url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    query = {
        'pageIndex' : i # page number
      }
    resp = requests.get(url, query)
    soup = BeautifulSoup(resp.content, 'lxml')
    tag = soup.select('div.fileGoupBox a') #tag
#     print(len(tag))
    total_list.extend(tag) 
    print(total_list)
    
    pdf_list = [] #.pdf를 가지는 리스트를 가져온다.
    for idx,t in enumerate(total_list):
      if total_list[idx].attrs['title'][-3:] == 'pdf': #.attrs['title']을 활용해서 title의 .pdf파일이 들어간 리스트를 뽑아낸다.
        pdf_list.append(total_list[idx]) 
    print(pdf_list)

[<a href="/portal/cmmn/file/fileDown.do?menuNo=200761&amp;atchFileId=KO_00000000000085857&amp;fileSn=1" title="2005년 제12차 금통위 의사록.hwp">
<i class="icon-down"></i> 2005년 제12차 금통위 의사록.hwp
												</a>, <a class="viewerLink" href="javascript:void(0);" onclick="fnDocViewer('portal','KO_00000000000085857', '1', 'B0000245' ); return false" rel="nofollow" title="새창열림"><span class="icon-viewer"><span class="hidden">문서보기 프로그램 새창으로 열립니다.</span></span></a>, <a href="/portal/cmmn/file/fileDown.do?menuNo=200761&amp;atchFileId=KO_00000000000085857&amp;fileSn=2" title="2005년 제12차 금통위 의사록 - PDF파일.pdf">
<i class="icon-down"></i> 2005년 제12차 금통위 의사록 - PDF파일.pdf
												</a>, <a class="viewerLink" href="javascript:void(0);" onclick="fnDocViewer('portal','KO_00000000000085857', '2', 'B0000245' ); return false" rel="nofollow" title="새창열림"><span class="icon-viewer"><span class="hidden">문서보기 프로그램 새창으로 열립니다.</span></span></a>]
[<a href="/portal/cmmn/file/fileDown.do?menuNo=200761&amp;atchFileId=KO_00000

In [ ]:
pdf_href_list = [] 
for i in pdf_list:
    pdf_href_list.append(i.attrs['href']) # .attrs['href']를 사용해서 url주소만 가져온다.
print(pdf_href_list)

site = 'https://www.bok.or.kr'

url = []
for i in range(len(pdf_href_list)): 
    url.append(site + pdf_href_list[i]) # site + url을 통해서 pdf파일이 다운 가능하게 한다.
print(url)

['/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085857&fileSn=2']
['https://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085857&fileSn=2']


In [ ]:
for i in range(len(url)): 
  mpb = requests.get(url[i]) #request.get을 통해 url을 저장
  with open('/content/gdrive/My Drive/MPB/{}.pdf'.format(i), 'wb') as f: #url파일들을 지정된 폴더로 다운받는다.
    f.write(mpb.content)

## tika(2005~2017 but 2006~7 data 제외) & fitz(2006~2007)이용해서 pdf to txt파일로 읽어오기

In [ ]:
pip install tika #tika parser이용!

In [ ]:
import re
from tika import parser #https://github.com/chrismattmann/tika-python 이용하였다.
import shutil
from os import listdir #listdir-directory에 있는 모든 파일을 가져온다
from os.path import isfile, join # isfile 지정된 경로의 파일명만 얻기
import os

def pdf2txt (source_folder, output_folder) :#pdf파일을 text파일로 변경해 주는 함수
    # 지정 폴더 내 파일 목록 조회 (파일만)
    # in ~~~ 는 폴더 내 다른 서브퓰더가 있는 경우를 처리하기 위함.
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))] 
    
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        
        try:    
            pdf_filepath = source_folder + pdf
            # pdf 파일을 text로 변환
            print(pdf_filepath)
            parsedPDF = parser.from_file(pdf_filepath)["content"]

        except Exception as e:
            parasedPDF = ' '
            print(f'{pdf} 에러 : {e}')
        
        ouput_filepath = (output_folder + pdf).replace('.pdf', '.txt')
        with open(ouput_filepath, 'w', encoding='utf-8') as f :
            print(ouput_filepath)
            f.write(parsedPDF)
            f.close()

In [ ]:
base = os.getcwd() # 현재 작업 폴더 얻기
source = os.path.join(base, '/content/gdrive/My Drive/') #Pdf파일 저장되어 있는 폴더
output = os.path.join(base, '/content/gdrive/My Drive/MPB_Text/')#Txt파일로 폴더 지정
pdf2txt(source, output) #pdf->text 파일로 변경

In [ ]:
!pip install PyMuPDF # 2006년도 2007년도 의사록 section이 뽑히지 않아서 fitz사용

In [ ]:
import fitz

In [ ]:
c_pdf = [] # 폴더에서 pdf파일을 불러와 리스트로 저장
for i in range(215, 261):
    c_pdf.append(fitz.open(r'C:\Users\junshey\jupyter\06-08 pdf\{}.pdf'.format(i)))

In [ ]:
for page in c_pdf[-1]:
    text = page.getText()
    print(to_str(text.encode("utf-8")))

In [ ]:
def to_str(bytes_or_str): # str로 변환 함수
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value   # str 인스턴스

In [ ]:
text_16_17 = [] 
for i, pdf in  enumerate(c_pdf): #
    print(pdf)
    full_text = ""
    with open(r'{}.txt'.format(i), "w", encoding='utf-8') as fout:
        for page in pdf:
            text = page.getText() # text로 불러오기

            fout.write(to_str(text.encode()))

           
            full_text += text
        print(full_text)
    text_16_17.append(full_text)

## section 2,3 데이터 가져오기

In [ ]:
k = [] # txt폴더에서 저장된 data 불러오기
for i in range(7,272):
    filename = r'C:\Users\junshey\jupyter\MPB_Text\{}.txt'.format(i)
    k.append(filename)
len(k)
print(k[0])

C:\Users\junshey\jupyter\MPB_Text\7.txt


In [ ]:
t = []
for i in range(len(k)):
    with open(k[i],'r',encoding='UTF-8') as file_object:
        contents = file_object.read()
        t.append(contents)
t[0]

'2017년도 제25차\n\n금융통화위원회(임시) 의사록\n\n한 국\n\n은 행\n\n\x0c1. 일\n\n시\n\n2017년 12월 28일(목)\n\n2. 장\n\n소\n\n금융통화위원회 회의실\n\n3. 출석위원\n\n이 주 열\n\n의 장(총재)\n\n함 준 호\n\n위 원\n\n이 일 형\n\n위 원\n\n조 동 철\n\n위 원\n\n고 승 범\n\n위 원\n\n신 인 석\n\n위 원\n\n윤 면 식\n\n위 원(부총재)\n\n4. 결석위원\n\n없\n\n음\n\n5. 참 여 자\n\n김 민 호\n\n부총재보\n\n임 형 준\n\n부총재보\n\n허 진 호\n\n부총재보\n\n전 승 철\n\n부총재보\n\n신 호 순\n\n부총재보\n\n서 봉 국\n\n외자운용원장\n\n장\n\n민\n\n조사국장\n\n손\n\n욱\n\n경제연구원장\n\n신\n\n운\n\n금융안정국장\n\n박 종 석\n\n통화정책국장\n\n이 환 석\n\n금융시장국장\n\n이 승 헌\n\n국제국장\n\n장 정 석\n\n금융통화위원회실장\n\n성 병 희\n\n공보관\n\n이 동 원\n\n의사팀장\n\n6. 회의경과\n\n<의안 제44호\n\n– 2018년 통화신용정책 운영방향(안)>\n\n(１) 담당 부총재보가「한국은행법」제6조에 의거 2018년 통화신용정책 운영\n방향을 수립 공표하고자 한다는 내용의 제안설명을 하였음.\n(２) 위원 토의내용\n제안설명에 이어 관련부서는 본 안건에 대해 주관위원의 의견을 반영한 수정\n안을 위원협의회에 보고한 바 있으며, 동 위원협의회에서는 다음과 같은 내용이\n논의되었다고 보고하였음.\n\n- 1 -\n\n\x0c먼저, 여러 위원들은 내년도 물가안정목표제 운영여건 점검 시 현행 중기 물가\n안정목표 수준 2%를 변경할 필요성은 크지 않아 보인다는 점이 고려될 필요가\n있다는 의견을 제시하였음.\n다음으로 일부 위원은 한국은행 대출제도 운영과 관련하여 대출제도의 합리\n화를 지속적으로 추진할 필요가 있다는 의견과 함께 향후 적격담보증권 설정과\n관련한 내

In [ ]:
import pandas as pd

In [ ]:
df_text = pd.DataFrame(t, columns=['text'])
df_text

,text
0,2017년도 제25차\n\n금융통화위원회(임시) 의사록\n\n한 국\n\n은 행\n...
1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,text\n금융통화위원회(정기) 의사록\n한 국 은 행\n- 1 -\n1. 일 시 ...
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...
260,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
261,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
262,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
263,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
import re 

def preprocess_minutes_s2(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

#     sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
#                 'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section2)


    return section_texts

In [ ]:
# [[],[],[]]형태로 먼저 변경
k= []
for i in df_text['text']:
    dd = preprocess_minutes_s2(i)
    k.append(dd)

section_2 = [] # 빈리스트는 ''값으로 추가해서 DataFrame변경
for i in range(len(k)): 
    if k[i] == []:
        section_2.append('') # []형태에는 dataframe이 만들어 지지 않기 때문에 ' 'NaN값의 형태로 변경한다.
    else:
        section_2.append(k[i])
        
section_2[:10]

['',
 '',
 '',
 ['일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를  인상할 것으로 예견되고 실물경기도 견조한 회복세를 지속하고 있음에도 불구하  고 수익률곡선은 계속 평탄화되고 있다고 지적하였음.',
  '일각에서는 미국의 장기실  질균형금리 및 기대인플레이션의 하락을 그 원인으로 거론하고 있지만 이 같은  요인은 이미 이전부터 장기금리에 반영되어 왔으므로 최근의 수익률곡선 변화를  설명하는 데에 한계가 있는 것으로 보인다고 언급하였음.',
  '한편 이러한 현상을 향  후 실물경기의 침체 우려와 글로벌 장기채권 수요의 유입에 따른 결과로 해석하  는 시각도 있다고 소개하고, 이에 대한 관련부서의 견해를 물었음.',
  '이에 대해 관련부서에서는 수익률곡선의 평탄화나 장단기금리차의 역전이 장  기금리의 하락에 의해 주도되는 경우에는 향후 실물경기의 침체를 예고하는 신호  (signal)로 받아들여질 수 있겠지만, 최근 미국의 수익률곡선 평탄화는 주로 재무  부의 단기채권 발행 확대 방침, 미 연준의 금리인상 기대 등으로 단기금리가 크  게 상승한 데에 따른 것으로 보인다고 답변하였음.',
  '이에 동 위원은 미국의 세제개편안이 의회를 통과한다면 수익률곡선이나 정  책금리 경로가 가팔라질 가능성은 없는지 관련부서의 견해를 물었으며,  이에 대해 관련부서에서는 이번 세제개편안이 법인세율 인하 등 대규모 감세  안을 골자로 하고 있는 만큼 미 금융시장 지표가 동 법안의 처리 관측에 따라 등  락하는 양상을 보이고 있다고 설명하였음.',
  '비록 일부 선반영된 측면도 있겠지만  미 의회에서 세제개편안을 가결한다면 재정적자 보전을 위한 장기국채의 발행 전  망이 부각되면서 수익률곡선이 가팔라질 가능성도 있을 것으로 생각된다고 답변    - 10 -  하였음.',
  '이와 관련하여 동 위원은 한·미 간 장기금리의 동조화 경향에도 불구하고 우  리나라의 수익률곡선이 미국과 달리 가팔라지고 있는 이유는 무엇인지 관련부서  의 

In [ ]:
df_section_2 = pd.DataFrame(section_2, columns=['section_2']) 
df_section_2

,section_2
0,
1,
2,
3,[일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인...
4,
...,...
260,[일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸...
261,[일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜...
262,
263,[일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도...


In [ ]:
import re 

def preprocess_minutes_s3(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

#     sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
#                 'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section3)


    return section_texts

In [ ]:
# [[],[],[]]형태로 먼저 변경
k= []
for i in df_text['text']:
    dd = preprocess_minutes_s3(i)
    k.append(dd)

section_3 = [] # 빈리스트는 ' '값으로 추가해서 DataFrame변경
for i in range(len(k)): 
    if k[i] == []: # []형태에는 dataframe이 만들어 지지 않기 때문에 ' 'NaN값의 형태로 변경한다.
        section_3.append('')
    else:
        section_3.append(k[i])
        
section_3[:10]

['', '', '', '', '', '', '', '', '', '']

In [ ]:
df_section_3 = pd.DataFrame(section_3, columns=['section_3']) 
df_section_3

,section_3
0,
1,
2,
3,
4,
...,...
260,[일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되...
261,[일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민...
262,
263,[일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지...


## 의사록 날짜 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup

date_list = [] # tag리스트를 저장
for i in range(5,33):
    url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    query = {
        'pageIndex' : i
      }
    resp = requests.get(url, query).text
    soup = BeautifulSoup(resp, 'html.parser') #resp.conetent 로 뽑으면 Tag형태로 나오기 때문에 content를 쓰지 않고 리스트 형태로 전환 (날짜만 뽑기 위해)
    tag = soup.select('div span.titlesub')
    date_list.extend(tag)
    
date_1 = [] # join함수를 써서 list->str로 변환 
for t in date_list:
  date_1.append('\n'.join(t))
len(date_1)

date_2 = date_1[7:-8] # 기존의 날짜로 정리(필요없는 날짜 슬라이스를 통해 정리)

date_3 = [] #0000.00.00 으로 가져오기 위해 split date_2를 수정
for d in range(len(date_2)): 
  date_3.append(date_2[d].split('('))

date_4 = [] #0000.00.00 으로 가져오기 위해 slice
for t in date_3:
  date_4.append(t[2][:-1])

date_5 = []
for d in date_4:
  if d[-1] == '.':
    date_5.append(d[:-1])
  else:
    date_5.append(d)

len(date_5)
date_5

['2017.12.28',
 '2017.12.14',
 '2017.12.7',
 '2017.11.30',
 '2017.11.9',
 '2017.10.19',
 '2017.9.21',
 '2017.8.31',
 '2017.8.10',
 '2017.7.13',
 '2017.6.22',
 '2017.5.25',
 '2017.4.27',
 '2017.4.13',
 '2017.3.23',
 '2017.2.23',
 '2017.1.26',
 '2017.1.13',
 '2016.12.29',
 '2016.12.21',
 '2016.12.15',
 '2016.11.24',
 '2016.11.11',
 '2016.10.27',
 '2016.10.13',
 '2016.9.22',
 '2016.9.9',
 '2016.8.11',
 '2016.7.14',
 '2016.7.1',
 '2016.6.23',
 '2016.6.9',
 '2016.5.13',
 '2016.4.19',
 '2016.3.24',
 '2016.3.10',
 '2016.2.25',
 '2016.2.16',
 '2016.1.28',
 '2016.1.14',
 '2015.12.24',
 '2015.12.16',
 '2015.12.10',
 '2015.11.26',
 '2015.11.12',
 '2015.10.29',
 '2015.10.15',
 '2015.9.24',
 '2015.9.11',
 '2015.8.27',
 '2015.8.13',
 '2015.7.23',
 '2015.7.9',
 '2015.6.25',
 '2015.6.11',
 '2015.5.15',
 '2015.4.23',
 '2015.4.9',
 '2015.3.26',
 '2015.3.12',
 '2015.2.17',
 '2015.1.29',
 '2015.1.15',
 '2014.12.24',
 '2014.12.11',
 '2014.11.13',
 '2014.10.23',
 '2014.10.15',
 '2014.9.25',
 '2014.9.12',
 '

In [ ]:
df_date = pd.DataFrame(date_5, columns=['date'])
df_date

,date
0,2017.12.28
1,2017.12.14
2,2017.12.7
3,2017.11.30
4,2017.11.9
...,...
260,2005.8.11
261,2005.7.7
262,2005.6.23
263,2005.6.9


In [ ]:
df = pd.concat([df_date,df_section_2, df_section_3], axis = 1) # 날짜, sec2, sec3 데이터를 concat 해준다.
df

,date,section_2,section_3
0,2017.12.28,,
1,2017.12.14,,
2,2017.12.7,,
3,2017.11.30,[일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인...,
4,2017.11.9,,
...,...,...,...
260,2005.8.11,[일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸...,[일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되...
261,2005.7.7,[일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜...,[일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민...
262,2005.6.23,,
263,2005.6.9,[일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도...,[일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지...


In [ ]:
df['section_3'] = [''.join(i) for i in df['section_3']] 
df['section_2'] = [''.join(i) for i in df['section_2']]
df # []지워준다.

,date,section_2,section_3
0,2017.12.28,,
1,2017.12.14,,
2,2017.12.7,,
3,2017.11.30,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인상...,
4,2017.11.9,,
...,...,...,...
260,2005.8.11,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...
261,2005.7.7,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...
262,2005.6.23,,
263,2005.6.9,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지,..."


In [ ]:
df['body'] = 0
df['body'] = df['section_2']+df['section_3'] # df column을 body를 생성해서 section2,3 데이터 합치기
df

,date,section_2,section_3,body
0,2017.12.28,,,
1,2017.12.14,,,
2,2017.12.7,,,
3,2017.11.30,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인상...,,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인상...
4,2017.11.9,,,
...,...,...,...,...
260,2005.8.11,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...
261,2005.7.7,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...
262,2005.6.23,,,
263,2005.6.9,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지,...",일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...


In [ ]:
for i in range(len(df)): # 빈리스트 행 지워주기 / dropna를 사용해도 괜춘할듯
    if df['section_2'][i] == '':
        if df['section_3'][i] == '':
            df = df.drop(i) 

In [ ]:
MIB_data=df[['date','body']]
MIB_data

,date,body
3,2017.11.30,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인상...
5,2017.10.19,일부 위원은 최근 미국에서는 임금상승세가 확대되고 세제개혁에 대한 기대 감이 커지...
7,2017.8.31,일부 위원은 미국의 금리 정상화 행보에도 불구하고 미 달러화가 약세기조를 지속하면...
9,2017.7.13,일부 위원은 주요국에서는 성장과 고용이 회복국면의 정점을 향해 가고 있음.에도 불구...
11,2017.5.25,일부 위원은 최근 국제금융시장이 안정된 모습을 지속하는 가운데 국내금융 시장도 외...
...,...,...
259,2005.9.8,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...
260,2005.8.11,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...
261,2005.7.7,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...
263,2005.6.9,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...


In [ ]:
# 피클로 저장 및 불러오기

import pickle

with open('MIB_data.pickle','wb') as f: #저장
    pickle.dump(df , f)
    
with open('BOK.pickle', 'rb') as f: #불러오기
    data = pickle.load(f)